<a href="https://colab.research.google.com/github/dinooooooi/dinooooi/blob/main/0429%EC%9B%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파머완 pg. 516~528

#07. 문서 군집화 소개와 실습
##문서 군집화 개념
: 비슷한 텍스트 구성의 문서를 군집화하는 것

##opinion review 데이터 세트를 이용한 문서 군집화 수행하기
https://archive.ics.uci.edu/dataset/191/opinosis+opinion+frasl+review

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import glob, os

# 다음은 저자의 컴퓨터에서 압축 파일을 풀어놓은 디렉터리이니, 각자 디렉터리를 다시 설정합니다.
path = r'/content/drive/MyDrive/OpinosisDataset1.0/topics'

# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합.
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

#개별 파일의 파일명은 filename_list로 취함.
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_test list로 취합
for file_ in all_files:
  # 개별 파일을 읽어서 DataFrame으로 생성
  df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')

  # 절대 경로로 주어진 파일명을 가공, 리눅스에서 수행할 때는 다음 \\를 \로 변경.
  # 맨 마지막 .data 확장자도 제거
  filename_ = file_.split('\\')[-1]
  filename = filename_.split(',')[0]

  # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가.
  filename_list.append(filename)
  opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...
1,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...
2,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...
3,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...
4,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#문서를 TF-IDF 형태로 피처 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                             ngram_range=(1,2), min_df=0.05, max_df=0.85)
# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [ ]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행.
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
1,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
2,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
3,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
4,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0


In [ ]:
# cluster_label=0인 데이터 세트 : 차량용 네비게이션에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
10,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
3,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
4,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
49,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
46,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
45,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
44,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
32,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
33,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0


In [ ]:
# cluster_label=1인 데이터 세트 : 포터블 전자기기에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
27,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
24,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
22,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
19,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
20,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
21,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
18,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
16,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
43,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1
2,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,1


In [ ]:
# cluster_label=2인 데이터 세트 : 자동차에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
28,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,2
29,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,2
30,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,2
6,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,2
42,/content/drive/MyDrive/OpinosisDataset1.0/topi...,headphone jack i got a clear case for it a...,2


In [ ]:
# cluster_label=3인 데이터 세트 : 대부분 호텔에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
25,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
23,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
13,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
17,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
9,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
14,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
7,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
1,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
41,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3
37,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,3


In [ ]:
# cluster_label=4인 데이터 세트 : 킨들 대한 리뷰로 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
11,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
12,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
15,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
8,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
48,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
5,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4
0,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,4


In [ ]:
# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
48,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
49,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
3,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
4,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
46,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
45,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
44,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
8,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0
33,/content/drive/MyDrive/OpinosisDataset1.0/topi...,...,0


##군집별 핵심 단어 추출하기

In [ ]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


In [ ]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
  cluster_details={}

  # cluster_centers array 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점 별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하며 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_.argsort()로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num,
                                                      top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

In [ ]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명 :', cluster_detail['filenames'][:7])
    print('===============================================')

In [ ]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num = 3, top_n_features = 10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['/content/drive/MyDrive/OpinosisDataset1.0/topics/price_holiday_inn_london.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/room_holiday_inn_london.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/rooms_bestwestern_hotel_sfo.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/location_holiday_inn_london.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/free_bestwestern_hotel_sfo.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/food_holiday_inn_london.txt.data', '/content/drive/MyDrive/OpinosisDataset1.0/topics/food_swissotel_chicago.txt.data']
####### Cluster 1
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 : ['/content/drive/MyDrive/OpinosisDataset1.0/topics/satellite_garmin_nuvi_255W_gps.txt.data', '/